In [1]:
import pandas as pd
from datetime import datetime

Co chcę zrobić?
* Uśrednić pogodowe dane godzinowe oraz dzienne do danych miesięcznych 🗹
* Stworzenie tabeli zawierającej datę, kod plantacji, Tiny+Small, Large+Mature oraz dane pogodowe 🗹
* Zamiana daty na dwie kolumny, rok oraz miesiąc 🗹
* One-hot encoding kodów plantacji, zatrzymanie mapowania w zmiennej do późniejszej operacji odwrotnej - ZROBIONE W SKRYPCIE Src\Models\TimeSeriesForecasters.py DLA KAŻDEGO RODZAJU MODELU 🗹
* Inżynieria cech, stworzenie lagów, sprawdzenie korelacji, ewentualna zaawansowana selekcja cech - ZROBIONE W SKRYPCIE Src\Models\feature_selection.ipynb 🗹

In [2]:
df_tbl_pods = pd.read_csv("tbl_pods.csv")
df_tbl_weather_daily = pd.read_csv("tbl_weather_daily.csv")
df_tbl_weather_hourly = pd.read_csv("tbl_weather_hourly.csv")

In [3]:
df_tbl_weather_daily.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1471612 entries, 0 to 1471611
Data columns (total 13 columns):
 #   Column                      Non-Null Count    Dtype  
---  ------                      --------------    -----  
 0   Date                        1471612 non-null  object 
 1   FarmID                      1471612 non-null  object 
 2   temperature_2m_max          1471612 non-null  float64
 3   temperature_2m_min          1471612 non-null  float64
 4   apparent_temperature_max    1471612 non-null  float64
 5   apparent_temperature_min    1471612 non-null  float64
 6   precipitation_sum           1471612 non-null  float64
 7   rain_sum                    1471612 non-null  float64
 8   precipitation_hours         1471612 non-null  float64
 9   windspeed_10m_max           1471612 non-null  float64
 10  windgusts_10m_max           1471612 non-null  float64
 11  shortwave_radiation_sum     1471612 non-null  float64
 12  et0_fao_evapotranspiration  1471612 non-null  float64
dt

In [4]:
df_tbl_weather_hourly.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35318688 entries, 0 to 35318687
Data columns (total 10 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   Date                           object 
 1   FarmID                         object 
 2   soil_temperature_0_to_7cm      float64
 3   soil_temperature_7_to_28cm     float64
 4   soil_temperature_28_to_100cm   float64
 5   soil_temperature_100_to_255cm  float64
 6   soil_moisture_0_to_7cm         float64
 7   soil_moisture_7_to_28cm        float64
 8   soil_moisture_28_to_100cm      float64
 9   soil_moisture_100_to_255cm     float64
dtypes: float64(8), object(2)
memory usage: 2.6+ GB


In [5]:
df_tbl_weather_daily['Date'] = pd.to_datetime(df_tbl_weather_daily['Date'])
df_tbl_weather_daily['month'] = pd.to_datetime(df_tbl_weather_daily['Date']).dt.month
df_tbl_weather_daily['year'] = pd.to_datetime(df_tbl_weather_daily['Date']).dt.year
df_tbl_weather_daily = df_tbl_weather_daily.groupby(['year','month', 'FarmID'],as_index=False).mean()
df_tbl_weather_daily

,year,month,FarmID,temperature_2m_max,temperature_2m_min,apparent_temperature_max,apparent_temperature_min,precipitation_sum,rain_sum,precipitation_hours,windspeed_10m_max,windgusts_10m_max,shortwave_radiation_sum,et0_fao_evapotranspiration
0,1998,1,AG10,34.741935,22.393548,36.770968,25.441935,0.429032,0.429032,0.903226,10.738710,24.787097,20.136774,4.689677
1,1998,1,AG11,34.229032,22.741935,36.709677,26.264516,0.448387,0.448387,0.806452,11.200000,26.074194,20.222258,4.645806
2,1998,1,AG12,34.548387,22.241935,36.729032,25.496774,0.429032,0.429032,0.903226,10.738710,24.787097,20.136774,4.648387
3,1998,1,AG13,34.232258,22.541935,36.938710,26.212903,0.374194,0.374194,0.967742,10.532258,24.393548,20.063226,4.497097
4,1998,1,AG130,34.503226,22.945161,37.161290,26.609677,0.422581,0.422581,1.000000,10.648387,24.874194,19.941613,4.520000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48501,2023,7,SP425,27.900000,22.600000,32.500000,26.400000,2.000000,2.000000,7.000000,9.350000,20.350000,11.715000,2.490000
48502,2023,7,SP426,27.800000,22.500000,32.300000,26.250000,2.000000,2.000000,7.000000,9.350000,20.350000,11.715000,2.485000
48503,2023,7,SP427,27.750000,22.450000,32.200000,26.100000,2.000000,2.000000,7.000000,9.350000,20.350000,11.715000,2.480000
48504,2023,7,SP428,27.250000,22.200000,32.050000,26.250000,8.450000,8.450000,10.000000,8.450000,23.750000,13.295000,2.700000


In [6]:
df_tbl_weather_hourly['Date'] = pd.to_datetime(df_tbl_weather_hourly['Date'])
df_tbl_weather_hourly['month'] = pd.to_datetime(df_tbl_weather_hourly['Date']).dt.month
df_tbl_weather_hourly['year'] = pd.to_datetime(df_tbl_weather_hourly['Date']).dt.year
df_tbl_weather_hourly = df_tbl_weather_hourly.groupby(['year','month', 'FarmID'],as_index=False).mean()
df_tbl_weather_hourly

,year,month,FarmID,soil_temperature_0_to_7cm,soil_temperature_7_to_28cm,soil_temperature_28_to_100cm,soil_temperature_100_to_255cm,soil_moisture_0_to_7cm,soil_moisture_7_to_28cm,soil_moisture_28_to_100cm,soil_moisture_100_to_255cm
0,1998,1,AG10,29.146371,28.788575,27.784005,26.701613,0.283711,0.301133,0.337141,0.367218
1,1998,1,AG11,27.960349,27.682796,26.942876,26.118414,0.245641,0.257746,0.298177,0.314079
2,1998,1,AG12,28.726478,28.301613,27.359274,26.438172,0.083534,0.098519,0.137215,0.168069
3,1998,1,AG13,28.073387,27.803091,27.038575,26.163441,0.318098,0.321200,0.362079,0.375735
4,1998,1,AG130,28.347849,28.072312,27.268414,26.330511,0.320082,0.320125,0.363851,0.376445
...,...,...,...,...,...,...,...,...,...,...,...
48501,2023,7,SP425,25.564583,25.766667,25.768750,26.700000,0.380271,0.380437,0.389979,0.331271
48502,2023,7,SP426,25.464583,25.666667,25.668750,26.600000,0.380271,0.380437,0.389979,0.331271
48503,2023,7,SP427,25.377083,25.581250,25.600000,26.500000,0.456458,0.453458,0.477062,0.436604
48504,2023,7,SP428,25.179167,25.425000,25.485417,26.300000,0.379063,0.377583,0.407187,0.401000


In [7]:
df_tbl_pods

,Date,FarmID,Tiny+Small,Large+Mature
0,1997-10-01,AG10,4.0,180.000000
1,1997-11-01,AG10,17.0,124.333333
2,1997-12-01,AG10,30.0,68.666667
3,1998-01-01,AG10,43.0,13.000000
4,1998-02-01,AG10,28.0,28.000000
...,...,...,...,...
9023,2023-03-01,MA413,2.0,11.000000
9024,2023-04-01,MA413,2.0,3.000000
9025,2023-05-01,MA413,3.0,4.000000
9026,2023-06-01,MA413,1.0,1.000000


In [8]:
# Dostępność czasowa danych pogodowych od 1998 roku, tak więc należy ograniczyć dane zbiorów
df_tbl_pods[df_tbl_pods["Date"] >= "1998-01-01"]

,Date,FarmID,Tiny+Small,Large+Mature
3,1998-01-01,AG10,43.0,13.0
4,1998-02-01,AG10,28.0,28.0
5,1998-03-01,AG10,13.0,43.0
6,1998-04-01,AG10,11.5,38.0
7,1998-05-01,AG10,10.0,33.0
...,...,...,...,...
9023,2023-03-01,MA413,2.0,11.0
9024,2023-04-01,MA413,2.0,3.0
9025,2023-05-01,MA413,3.0,4.0
9026,2023-06-01,MA413,1.0,1.0


In [9]:
df_tbl_weather_daily['Date'] = [datetime(row["year"], row["month"], 1) for index, row in df_tbl_weather_daily.iterrows()]
df_tbl_weather_hourly['Date'] = [datetime(row["year"], row["month"], 1) for index, row in df_tbl_weather_hourly.iterrows()]

In [10]:
df_tbl_weather_daily.drop(columns=["year", "month"], inplace=True)

In [11]:
df_tbl_pods['Date'] = pd.to_datetime(df_tbl_pods['Date'])

In [12]:
df_temp = pd.merge(df_tbl_pods[df_tbl_pods["Date"] >= "1998-01-01"], df_tbl_weather_daily, on=["Date", "FarmID"])

In [13]:
df_temp

,Date,FarmID,Tiny+Small,Large+Mature,temperature_2m_max,temperature_2m_min,apparent_temperature_max,apparent_temperature_min,precipitation_sum,rain_sum,precipitation_hours,windspeed_10m_max,windgusts_10m_max,shortwave_radiation_sum,et0_fao_evapotranspiration
0,1998-01-01,AG10,43.0,13.0,34.741935,22.393548,36.770968,25.441935,0.429032,0.429032,0.903226,10.738710,24.787097,20.136774,4.689677
1,1998-02-01,AG10,28.0,28.0,35.189286,24.317857,39.117857,28.378571,1.857143,1.857143,1.535714,11.317857,25.207143,20.272143,4.714643
2,1998-03-01,AG10,13.0,43.0,35.725806,25.367742,40.496774,29.983871,1.654839,1.654839,2.354839,13.103226,30.777419,20.438065,4.916774
3,1998-04-01,AG10,11.5,38.0,33.100000,24.903333,39.120000,29.943333,4.073333,4.073333,4.100000,12.033333,29.463333,17.877667,4.014333
4,1998-05-01,AG10,10.0,33.0,32.254839,24.303226,37.919355,29.096774,2.603226,2.603226,3.741935,11.693548,28.103226,17.486452,3.876129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9011,2023-03-01,MA413,2.0,11.0,32.625806,23.593548,37.380645,27.251613,2.758065,2.758065,3.709677,12.983871,29.116129,19.784194,4.540323
9012,2023-04-01,MA413,2.0,3.0,32.833333,23.786667,37.903333,27.520000,2.563333,2.563333,3.933333,12.423333,28.396667,20.378667,4.632667
9013,2023-05-01,MA413,3.0,4.0,32.225806,24.061290,37.435484,28.312903,3.029032,3.029032,4.903226,10.941935,25.816129,18.983226,4.261290
9014,2023-06-01,MA413,1.0,1.0,29.463333,22.926667,33.333333,26.320000,6.823333,6.823333,9.066667,12.590000,26.870000,14.725667,3.271000


In [14]:
df_temp = pd.merge(df_temp, df_tbl_weather_hourly, on=["Date", "FarmID"])

In [15]:
df_temp

,Date,FarmID,Tiny+Small,Large+Mature,temperature_2m_max,temperature_2m_min,apparent_temperature_max,apparent_temperature_min,precipitation_sum,rain_sum,...,year,month,soil_temperature_0_to_7cm,soil_temperature_7_to_28cm,soil_temperature_28_to_100cm,soil_temperature_100_to_255cm,soil_moisture_0_to_7cm,soil_moisture_7_to_28cm,soil_moisture_28_to_100cm,soil_moisture_100_to_255cm
0,1998-01-01,AG10,43.0,13.0,34.741935,22.393548,36.770968,25.441935,0.429032,0.429032,...,1998,1,29.146371,28.788575,27.784005,26.701613,0.283711,0.301133,0.337141,0.367218
1,1998-02-01,AG10,28.0,28.0,35.189286,24.317857,39.117857,28.378571,1.857143,1.857143,...,1998,2,30.208780,29.864286,28.965923,27.353274,0.351561,0.334216,0.317765,0.362451
2,1998-03-01,AG10,13.0,43.0,35.725806,25.367742,40.496774,29.983871,1.654839,1.654839,...,1998,3,31.365054,31.121774,30.227554,28.187097,0.338056,0.309485,0.305081,0.357526
3,1998-04-01,AG10,11.5,38.0,33.100000,24.903333,39.120000,29.943333,4.073333,4.073333,...,1998,4,29.077639,29.184167,29.357361,28.793472,0.412431,0.368989,0.298226,0.352946
4,1998-05-01,AG10,10.0,33.0,32.254839,24.303226,37.919355,29.096774,2.603226,2.603226,...,1998,5,28.380780,28.423656,28.578495,28.817876,0.429190,0.411262,0.310177,0.349663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9011,2023-03-01,MA413,2.0,11.0,32.625806,23.593548,37.380645,27.251613,2.758065,2.758065,...,2023,3,28.529167,28.526478,28.419355,27.342876,0.390527,0.343621,0.305570,0.424620
9012,2023-04-01,MA413,2.0,3.0,32.833333,23.786667,37.903333,27.520000,2.563333,2.563333,...,2023,4,28.858889,28.759306,28.444028,27.731250,0.398275,0.348147,0.299015,0.413540
9013,2023-05-01,MA413,3.0,4.0,32.225806,24.061290,37.435484,28.312903,3.029032,3.029032,...,2023,5,28.559140,28.523656,28.383737,27.991801,0.416329,0.391871,0.301789,0.404790
9014,2023-06-01,MA413,1.0,1.0,29.463333,22.926667,33.333333,26.320000,6.823333,6.823333,...,2023,6,26.370000,26.618611,27.324167,27.972361,0.454496,0.444658,0.349479,0.399222


In [16]:
df_temp['month'] = pd.to_datetime(df_temp['Date']).dt.month
df_temp['year'] = pd.to_datetime(df_temp['Date']).dt.year
df_temp.drop(columns=["Date"], inplace=True)

In [17]:
df_temp = df_temp[["FarmID", "year", "month", "Tiny+Small", "Large+Mature", "temperature_2m_max", "temperature_2m_min", 
"apparent_temperature_max", "apparent_temperature_min", "precipitation_sum", "rain_sum", 
"precipitation_hours", "windspeed_10m_max", "windgusts_10m_max", "shortwave_radiation_sum", 
"et0_fao_evapotranspiration", "soil_temperature_0_to_7cm", "soil_temperature_7_to_28cm",
"soil_temperature_28_to_100cm", "soil_temperature_100_to_255cm", "soil_moisture_0_to_7cm", "soil_moisture_7_to_28cm", 
"soil_moisture_28_to_100cm", "soil_moisture_100_to_255cm"]]

In [18]:
df_temp.to_csv("prepared_data.csv", index=False)